# HW


> 설명

다음은 tensorflow 에서 제공하는 pre-trained 모델들이다.

https://keras.io/api/applications/

ResNet50v2 모델은 수업에서 설명한 ResNet 모델에서 BatchNormalization 과 ReLu 의 순서를 변경하여 약간의 성능 이득을 달성한 버전이다.

<img src="https://www.researchgate.net/profile/Ngo-Le-Huy-Hien/publication/342334669/figure/fig2/AS:904483026513920@1592656966286/A-comparison-between-ResNet-v1-and-ResNet-v2-on-residual-blocks-23.ppm" width=400 />



> 문제

ResNet50v2 모델을 이용하여 Cifar10 데이터셋을 분류하는 모델을 Transfer Learning 방법으로 구현하시오. 

* model.summary() 로 결과를 출력하시오.
* epoch 1 로 모델을 학습하여 정확도를 출력하시오.
* HINT: 교재 p405 를 참고하시오.

> 제출

PLATO "(평가) 실습 과제 제출 게시판"에 Colab > 파일 > 다운로드 .ipynb 한 파일을 비밀글로 제출하시오.


In [6]:
'''
ref1: https://keras.io/applications/
ref2:https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet_common.py
ref3:
https://github.com/keras-team/keras-applications/releases/tag/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
'''
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image  # pip install pillow

import numpy as np
import matplotlib.pyplot as plt

#1:
##import tensorflow as tf
##gpus = tf.config.experimental.list_physical_devices('GPU')
##tf.config.experimental.set_memory_growth(gpus[0], True)


#2
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')  # (50000, 32, 32, 3)
x_test = x_test.astype('float32')  # (10000, 32, 32, 3)

# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# preprocessing, 'caffe', x_train, x_test: BGR
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

#3: resize_layer
inputs = Input(shape=(32, 32, 3))
resize_layer = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (224, 224)))(inputs)
res_model = ResNet50V2(weights='imagenet', include_top=False, input_tensor=resize_layer)
res_model.trainable = False

#4: create top for cifar10 classification
x = res_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outs = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outs)
model.summary()

#5: train and evaluate the model
#filepath = "RES/ckpt/4603-model.h5"
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 0, save_best_only = True)

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
ret = model.fit(x_train, y_train, epochs=1, batch_size=64, validation_split=0.3,
                verbose=2)  #, callbacks = [cp_callback])

y_pred = model.predict(x_train)
y_label = np.argmax(y_pred, axis=1)
C = tf.math.confusion_matrix(np.argmax(y_train, axis=1), y_label)
print("confusion_matrix(C):", C)
train_loss, train_acc = model.evaluate(x_train, y_train, verbose=2)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

#6: plot accuracy and loss
fig, ax = plt.subplots(1, 2, figsize=(10, 6))
ax[0].plot(ret.history['loss'], "g-")
ax[0].set_title("train loss")
ax[0].set_xlabel('epochs')
ax[0].set_ylabel('loss')

ax[1].set_ylim(0, 1.1)
ax[1].plot(ret.history['accuracy'], "b-", label="train accuracy")
ax[1].plot(ret.history['val_accuracy'], "r-", label="val_accuracy")
ax[1].set_title("accuracy")
ax[1].set_xlabel('epochs')
ax[1].set_ylabel('accuracy')
plt.legend(loc='lower right')
fig.tight_layout()
plt.show()

94668760/94668760 [==============================] - 10s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['lambda[0][0]']                 
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                            

KeyboardInterrupt: 